<a href="https://colab.research.google.com/github/Kampus-Merdeka-Software-Engineering/BE-2-Jakarta-28/blob/main/SmartFishPond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os

# Cek apakah file 'credentials.json' ada di direktori saat ini
print("Is 'credentials.json' file available:", os.path.isfile('credentials.json'))


Is 'credentials.json' file available: False


In [13]:
import tensorflow as tf
import numpy as np
import firebase_admin
from firebase_admin import credentials, db
from time import sleep

# Inisialisasi Firebase
cred = credentials.Certificate('/content/smart-fish-pond-d04cb-firebase-adminsdk-hcmv9-c41f694157.json')

# Cek apakah Firebase sudah diinisialisasi sebelumnya
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://smart-fish-pond-d04cb-default-rtdb.firebaseio.com/'
    })
else:
    print("Firebase sudah diinisialisasi sebelumnya.")

ref = db.reference('/sensor_turbidity')

# Membuat fungsi untuk mendapatkan data turbidity dari Firebase
def get_turbidity_data():
    try:
        data = ref.get()
        if data is not None and 'turbidity' in data:
            return data['turbidity']
        else:
            print("Data turbidity tidak tersedia atau tidak valid.")
            return None
    except Exception as e:
        print("Terjadi kesalahan saat mengambil data turbidity:", str(e))
        return None

# Membuat fungsi untuk mengontrol servo
def control_servo(delay):
    try:
        # Kontrol servo di ESP32 dengan delay sesuai prediksi
        print(f"Predicted Servo Delay: {delay}")
        sleep(delay)
    except Exception as e:
        print("Terjadi kesalahan saat mengontrol servo:", str(e))

# Membuat fungsi untuk menjalankan prediksi dan kontrol servo secara real-time
def run_realtime_prediction(model, X_train, y_train):
    while True:
        turbidity_value = get_turbidity_data()
        if turbidity_value is not None and len(turbidity_value) > 0:
            turbidity_value_normalized = turbidity_value / np.max(X_train)
            servo_delay_normalized = model.predict(np.array([[turbidity_value_normalized]]))[0][0]
            servo_delay = int(servo_delay_normalized * np.max(y_train))
            control_servo(servo_delay)
        else:
            # Tunggu sebelum mencoba lagi
            sleep(5)

# Mendapatkan data training dari Firebase
data_snapshot = ref.get()
X_train = []
y_train = []
if data_snapshot:
    for key, value in data_snapshot.items():
        # Pastikan value adalah dictionary sebelum mencoba mengakses key
        if isinstance(value, dict):
            turbidity_value = value.get('turbidity')
            servo_delay = value.get('servo_delay')
            if turbidity_value is not None and servo_delay is not None:
                X_train.append(turbidity_value)
                y_train.append(servo_delay)
        else:
            print(f"Data tidak valid: {value}")

# Konversi data ke numpy arrays
if X_train and y_train:
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # Normalisasi data
    X_train = X_train / np.max(X_train)
    y_train = y_train / np.max(y_train)

    # Membangun model menggunakan TensorFlow
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(1,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Melatih model
    model.fit(X_train, y_train, epochs=100, verbose=0)

    # Menjalankan prediksi dan kontrol servo secara real-time
    run_realtime_prediction(model, X_train, y_train)
else:
    print("Tidak ada data yang valid ditemukan untuk dilatih.")


Firebase sudah diinisialisasi sebelumnya.
Data tidak valid: 0
Data tidak valid: 0
Tidak ada data yang valid ditemukan untuk dilatih.


In [3]:
from google.colab import files

# Upload credentials.json file
uploaded = files.upload()


Saving smart-fish-pond-d04cb-firebase-adminsdk-hcmv9-c41f694157.json to smart-fish-pond-d04cb-firebase-adminsdk-hcmv9-c41f694157.json
